In [1]:
# import tensorflow as tf
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.utils import frame_utils
import cv2
import os
import numpy as np
import time

import zlib

import tensorflow.compat.v2 as tf
from pyquaternion import Quaternion

from waymo_open_dataset import dataset_pb2
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
tf.enable_v2_behavior()

2024-07-25 00:50:44.166024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-25 00:50:44.295827: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 00:50:45.256528: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rajeev-gupta/sensyn_ws/devel/lib:/opt/ros/noetic/lib:/media/rajeev-gupta/Drive250/conda_envs

In [3]:
record_files = ['raw_data/segment-13182548552824592684_4160_250_4180_250_with_camera_labels.tfrecord',
                'raw_data/segment-13145971249179441231_1640_000_1660_000_with_camera_labels.tfrecord',
                'raw_data/segment-12974838039736660070_4586_990_4606_990_with_camera_labels.tfv2v2v2v2v2v2v2v2record']

In [4]:
def extract_calibration_matrices(frame):
    calibrations = {}
    for camera in frame.context.camera_calibrations:
        intrinsic = np.array(camera.intrinsic).reshape(3, 3)
        extrinsic = np.array(camera.extrinsic.transform).reshape(4, 4)
        calibrations[camera.name] = {
            "intrinsic": intrinsic,
            "extrinsic": extrinsic
        }
    # print(calibrations)
    return calibrations

def convert_to_kitti_format(calibrations):
    kitti_format = {}
    for camera_name, params in calibrations.items():
        P = np.zeros((3, 4))
        P[:, :3] = params["intrinsic"]
        kitti_format[f'P{camera_name-1}'] = P
        kitti_format[f'Tr_velo_to_cam{camera_name-1}'] = params['extrinsic'][:3, :]
    kitti_format[f'R0_rect'] = np.eye(3)
    return kitti_format

def save_calibration_to_file(kitti_format, index, calib_folder):
    file_path = os.path.join(calib_folder, f'{index:06d}.txt')
    with open(file_path, 'w') as f:
        for key, matrix in kitti_format.items():
            f.write(f'{key}: {" ".join(map(str, matrix.flatten()))}\n')

def save_calib_to_kitti(frame, index, calib_folder):
    calibrations = extract_calibration_matrices(frame)
    kitti_format = convert_to_kitti_format(calibrations)
    save_calibration_to_file(kitti_format, index, calib_folder)
    
    
def process_tfrecord(file_path, calib_folder):
    if not os.path.exists(calib_folder):
        os.makedirs(calib_folder)
    
    dataset = tf.data.TFRecordDataset(file_path, compression_type='')

    for index, data in enumerate(dataset):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        
        save_calib_to_kitti(frame, index, calib_folder)
        
    print(f"Calibration files saved in {calib_folder}")


In [5]:
# process_tfrecord(tfrecord_path, 'kitti/calib')

In [6]:
CAMERA_NAME_MAP = {
    open_dataset.CameraName.FRONT: 'FRONT',
    open_dataset.CameraName.FRONT_LEFT: 'FRONT_LEFT',
    open_dataset.CameraName.FRONT_RIGHT: 'FRONT_RIGHT',
    open_dataset.CameraName.SIDE_LEFT: 'SIDE_LEFT',
    open_dataset.CameraName.SIDE_RIGHT: 'SIDE_RIGHT'
}

def save_images_to_kitti_format(frame, index, base_image_folder):
    for camera_image in frame.images:
        camera_name = CAMERA_NAME_MAP[camera_image.name]
        image_folder = os.path.join(base_image_folder, camera_name)
        if not os.path.exists(image_folder):
            os.makedirs(image_folder)
        
        image_filename = f'{index:06d}.png'
        image_path = os.path.join(image_folder, image_filename)

        # Decode the image using OpenCV
        img = tf.image.decode_jpeg(camera_image.image).numpy()
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(image_path, img)

def process_tfrecord_images(file_path, base_image_folder):
    if not os.path.exists(base_image_folder):
        os.makedirs(base_image_folder)

    dataset = tf.data.TFRecordDataset(file_path, compression_type='')

    for index, data in enumerate(dataset):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))

        save_images_to_kitti_format(frame, index, base_image_folder)

    print(f"Images saved in {base_image_folder}")


In [7]:

# process_tfrecord_images(tfrecord_path, 'kitti/images')


In [ ]:

def extract_points_from_range_image(laser, calibration, frame_pose):
  """Decode points from lidar."""
  if laser.name != calibration.name:
    raise ValueError('Laser and calibration do not match')
  if laser.name == dataset_pb2.LaserName.TOP:
    frame_pose = tf.convert_to_tensor(
        np.reshape(np.array(frame_pose.transform), [4, 4]))
    range_image_top_pose = dataset_pb2.MatrixFloat.FromString(
        zlib.decompress(laser.ri_return1.range_image_pose_compressed))
    # [H, W, 6]
    range_image_top_pose_tensor = tf.reshape(
        tf.convert_to_tensor(range_image_top_pose.data),
        range_image_top_pose.shape.dims)
    # [H, W, 3, 3]
    range_image_top_pose_tensor_rotation = transform_utils.get_rotation_matrix(
        range_image_top_pose_tensor[..., 0],
        range_image_top_pose_tensor[..., 1], range_image_top_pose_tensor[...,
                                                                         2])
    range_image_top_pose_tensor_translation = range_image_top_pose_tensor[...,
                                                                          3:]
    range_image_top_pose_tensor = transform_utils.get_transform(
        range_image_top_pose_tensor_rotation,
        range_image_top_pose_tensor_translation)
    frame_pose = tf.expand_dims(frame_pose, axis=0)
    pixel_pose = tf.expand_dims(range_image_top_pose_tensor, axis=0)
  else:
    pixel_pose = None
    frame_pose = None
  first_return = zlib.decompress(
      laser.ri_return1.range_image_compressed)
  second_return = zlib.decompress(
      laser.ri_return2.range_image_compressed)
  points_list = []
  for range_image_str in [first_return, second_return]:
    range_image = dataset_pb2.MatrixFloat.FromString(range_image_str)
    if not calibration.beam_inclinations:
      beam_inclinations = range_image_utils.compute_inclination(
          tf.constant([
              calibration.beam_inclination_min, calibration.beam_inclination_max
          ]),
          height=range_image.shape.dims[0])
    else:
      beam_inclinations = tf.constant(calibration.beam_inclinations)
    beam_inclinations = tf.reverse(beam_inclinations, axis=[-1])
    extrinsic = np.reshape(np.array(calibration.extrinsic.transform), [4, 4])
    range_image_tensor = tf.reshape(
        tf.convert_to_tensor(range_image.data), range_image.shape.dims)
    range_image_mask = range_image_tensor[..., 0] > 0
    range_image_cartesian = (
        range_image_utils.extract_point_cloud_from_range_image(
            tf.expand_dims(range_image_tensor[..., 0], axis=0),
            tf.expand_dims(extrinsic, axis=0),
            tf.expand_dims(tf.convert_to_tensor(beam_inclinations), axis=0),
            pixel_pose=pixel_pose,
            frame_pose=frame_pose))
    range_image_cartesian = tf.squeeze(range_image_cartesian, axis=0)
    points_tensor = tf.gather_nd(
        tf.concat([range_image_cartesian, range_image_tensor[..., 1:4]],
                  axis=-1),
        tf.where(range_image_mask))
    points_list.append(points_tensor.numpy())
  return points_list



def extract_points(lasers, laser_calibrations, frame_pose):
  """Extract point clouds."""
  sort_lambda = lambda x: x.name
  lasers_with_calibration = zip(
      sorted(lasers, key=sort_lambda),
      sorted(laser_calibrations, key=sort_lambda))
  points_xyz = []
  points_feature = []
  points_nlz = []
  for laser, calibration in lasers_with_calibration:
    points_list = extract_points_from_range_image(laser, calibration,
                                                  frame_pose)
    points = np.concatenate(points_list, axis=0)
    points_xyz.extend(points[..., :3].astype(np.float32))
    points_feature.extend(points[..., 3:5].astype(np.float32))
    points_nlz.extend(points[..., 5].astype(np.float32))
  return {
      'points_xyz': np.asarray(points_xyz),
      'points_feature': np.asarray(points_feature),
  }
def decode_frame(frame, frame_id):
  """Decodes native waymo Frame proto to tf.Examples."""

  lidars = extract_points(frame.lasers,
                          frame.context.laser_calibrations,
                          frame.pose)

  frame_name = '{scene_name}_{location}_{time_of_day}_{timestamp}'.format(
      scene_name=frame.context.name,
      location=frame.context.stats.location,
      time_of_day=frame.context.stats.time_of_day,
      timestamp=frame.timestamp_micros)

  example_data = {
      'scene_name': frame.context.name,
      'frame_name': frame_name,
      'frame_id': frame_id,
      'lidars': lidars,
  }

  return example_data

def extract_and_save_point_cloud(frame, frame_id, base_folder):
    decoded_frame = decode_frame(frame, frame_id)
    points_xyz = decoded_frame['lidars']['points_xyz']
    points_feature = decoded_frame['lidars']['points_feature']
    points_all = np.concatenate((points_xyz, points_feature), axis=1)
    bin_filename = f'{frame_id:06d}.bin'
    file_folder = base_folder
    if not os.path.exists(file_folder):
        os.makedirs(file_folder)
    bin_filepath = os.path.join(file_folder, bin_filename)
    points_all.astype(np.float32).tofile(bin_filepath)



In [9]:
# process_tfrecord_point_clouds(tfrecord_path, 'kitti/velodyne')


In [10]:
def process_tfrecord(file_path, base_calib_folder, base_image_folder, base_point_folder):
    base_folders = [base_calib_folder, base_image_folder, base_point_folder]
    for base_folder in base_folders:
        if not os.path.exists(base_folder):
            os.makedirs(base_folder)
    dataset = tf.data.TFRecordDataset(file_path, compression_type='')
    for index, data in enumerate(dataset):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        print('----------header-----------\n \
              Extracting data')
        extract_and_save_point_cloud(frame, index, base_point_folder)
        print('Point clouds done for index: ', index)
        save_images_to_kitti_format(frame, index, base_image_folder)
        print('Images done for index: ', index)
        save_calib_to_kitti(frame, index, base_calib_folder)
        print('Calibration done for index: ', index, '\n')

    print(f"done")


In [11]:
# process_tfrecord(tfrecord_path, 'kitti/calib', 'kitti/images', 'kitti/velodyne')

In [12]:
def convert_scenes_from_waymo_to_kitti(tfrecord_array, root_path, sleep_time= 5):
    for i, tfrecord_path in enumerate(tfrecord_array):
        scene_folder = os.path.join(root_path, tfrecord_path[:-10])
        if not os.path.exists(scene_folder):
            os.makedirs(scene_folder)
        calib_dir = os.path.join(scene_folder, 'calib')
        image_dir = os.path.join(scene_folder, 'image')
        velodyne_dir = os.path.join(scene_folder, 'velodyne')
        process_tfrecord(tfrecord_path, calib_dir, image_dir, velodyne_dir)
        print('sleeping for ', sleep_time)
        time.sleep(sleep_time)

In [13]:
parent_path = ''
convert_scenes_from_waymo_to_kitti(tfrecord_array=os.path.join(parent_path, record_files), root_path='waymo')

2024-07-25 00:50:46.800942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-25 00:50:46.834692: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-25 00:50:46.838060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-25 00:50:46.841548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

----------header-----------
               Extracting data
Calibration done for index:  0 

----------header-----------
               Extracting data
Calibration done for index:  1 

----------header-----------
               Extracting data
Calibration done for index:  2 

----------header-----------
               Extracting data
Calibration done for index:  3 

----------header-----------
               Extracting data
Calibration done for index:  4 

----------header-----------
               Extracting data
Calibration done for index:  5 

----------header-----------
               Extracting data
Calibration done for index:  6 

----------header-----------
               Extracting data
Calibration done for index:  7 

----------header-----------
               Extracting data
Calibration done for index:  8 

----------header-----------
               Extracting data
Calibration done for index:  9 

----------header-----------
               Extracting data
Calibration done for 

: 